## Import of data related to trips from and to Paris with shared vehicles.

### Libs imports.

In [1]:
import pandas as pd
import requests
import json

### Connecting directly to the csv file on data.gouv.fr (and testing the file).

In [2]:
# test url
url_2401 = 'https://static.data.gouv.fr/resources/trajets-realises-en-covoiturage-registre-de-preuve-de-covoiturage/20240208-001343/2024-01.csv'

# raw reading of the file to see what separators are used in the file
r = requests.get(url_2401)
print(r.text[:1000]) 


"journey_id";"trip_id";"journey_start_datetime";"journey_start_date";"journey_start_time";"journey_start_lon";"journey_start_lat";"journey_start_insee";"journey_start_department";"journey_start_town";"journey_start_towngroup";"journey_start_country";"journey_end_datetime";"journey_end_date";"journey_end_time";"journey_end_lon";"journey_end_lat";"journey_end_insee";"journey_end_department";"journey_end_town";"journey_end_towngroup";"journey_end_country";"passenger_seats";"operator_class";"journey_distance";"journey_duration";"has_incentive"
"21223205";"7ebf6533-a58c-4367-9a11-574ce726e5f5";"2024-01-01T00:00:00+01:00";"2024-01-01";"00:00:00";"2.351";"48.810";"94043";"94";"Le Kremlin-BicÃªtre";"MÃ©tropole du Grand Paris";"France";"2024-01-01T00:30:00+01:00";"2024-01-01";"00:30:00";"2.372";"48.953";"93072";"93";"Stains";"MÃ©tropole du Grand Paris";"France";"1";"C";"23398";"29";"OUI"
"21213421";"970d5c9b-9717-4f5f-8418-7dd00a0425ee";"2024-01-01T00:00:00+01:00";"2024-01-01";"00:00:00";"2.198

In [3]:
# reading the csv file and checking some details about it
df = pd.read_csv(url_2401, sep=";", low_memory=False)
df = df.dropna(subset=["journey_id", "journey_start_date", "journey_start_department", "journey_start_town", "journey_end_department", "journey_end_town", "passenger_seats", "journey_distance", "journey_duration"])

print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 896309 entries, 0 to 941039
Data columns (total 27 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   journey_id                896309 non-null  int64  
 1   trip_id                   896309 non-null  object 
 2   journey_start_datetime    896309 non-null  object 
 3   journey_start_date        896309 non-null  object 
 4   journey_start_time        896309 non-null  object 
 5   journey_start_lon         896309 non-null  float64
 6   journey_start_lat         896309 non-null  float64
 7   journey_start_insee       896309 non-null  object 
 8   journey_start_department  896309 non-null  object 
 9   journey_start_town        896309 non-null  object 
 10  journey_start_towngroup   896309 non-null  object 
 11  journey_start_country     896309 non-null  object 
 12  journey_end_datetime      896309 non-null  object 
 13  journey_end_date          896309 non-null  object

### Only keeping useful data and columns (Paris-related trips, journey distance, number of passengers, etc.)

In [4]:
df = df[(df["journey_start_department"] == "75") | (df["journey_end_department"] == "75")] 
df = df[["journey_id", "journey_start_date", "journey_start_department", "journey_start_town", "journey_end_department", "journey_end_town", "passenger_seats", "journey_distance", "journey_duration"]]
df = df[df["journey_duration"] >= 10]
df["journey_start_department"] = df["journey_start_department"].astype(int)
df["journey_end_department"] = df["journey_end_department"].astype(int)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 19167 entries, 60 to 941029
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   journey_id                19167 non-null  int64 
 1   journey_start_date        19167 non-null  object
 2   journey_start_department  19167 non-null  int64 
 3   journey_start_town        19167 non-null  object
 4   journey_end_department    19167 non-null  int64 
 5   journey_end_town          19167 non-null  object
 6   passenger_seats           19167 non-null  int64 
 7   journey_distance          19167 non-null  int64 
 8   journey_duration          19167 non-null  int64 
dtypes: int64(6), object(3)
memory usage: 1.5+ MB
None


## Going for the full set of csv files from 2019 to 2024, fusing them into a single csv file.

### Loading the json containing the manually-scraped urls pointing to all needed csv files.

In [5]:
with open("registre-preuve-covoiturage_Data_Gouv_urls.json", "r") as f:
    urls = json.load(f)


### Looping through csv for all available years and months 

In [6]:
# List to accumulate dataframes
dfs = []

for key in urls:
    url = urls[key]
    print(f"Processing {key}...")
    df = pd.read_csv(url, sep=";", low_memory=False)
    # Filtering Paris
    df = df[(df["journey_start_department"] == "75") | (df["journey_end_department"] == "75")] 
    # Filtering columns
    df = df[["journey_id", "journey_start_date", "journey_start_department", "journey_start_town", "journey_end_department", "journey_end_town", "passenger_seats", "journey_distance", "journey_duration"]]
    #droping NA rows
    df = df.dropna()
    # Filtering unrelevant trips (duration < 5 min) 
    df = df[df["journey_duration"] >= 5]
    # Changing some types
    df["journey_start_department"] = df["journey_start_department"].astype(int)
    df["journey_end_department"] = df["journey_end_department"].astype(int)
    # Adding to the list
    dfs.append(df)

# Concat all DataFrames in one go
df_final = pd.concat(dfs, ignore_index=True)
    
    

Processing 2019-02...
Processing 2019-03...
Processing 2019-04...
Processing 2019-05...
Processing 2019-06...
Processing 2019-07...
Processing 2019-08...
Processing 2019-09...
Processing 2019-10...
Processing 2019-11...
Processing 2019-12...
Processing 2020-01...
Processing 2020-02...
Processing 2020-03...
Processing 2020-04...
Processing 2020-05...
Processing 2020-06...
Processing 2020-07...
Processing 2020-08...
Processing 2020-09...
Processing 2020-10...
Processing 2020-11...
Processing 2020-12...
Processing 2021-01...
Processing 2021-02...
Processing 2021-03...
Processing 2021-04...
Processing 2021-05...
Processing 2021-06...
Processing 2021-07...
Processing 2021-08...
Processing 2021-09...
Processing 2021-10...
Processing 2021-11...
Processing 2021-12...
Processing 2022-01...
Processing 2022-02...
Processing 2022-03...
Processing 2022-04...
Processing 2022-05...
Processing 2022-06...
Processing 2022-07...
Processing 2022-08...
Processing 2022-09...
Processing 2022-10...
Processing

### Checking the result

In [7]:
print(df_final.head())
print(df_final.info())

   journey_id journey_start_date  journey_start_department  \
0     7226722         2022-06-01                        75   
1     7243957         2022-06-01                        75   
2     7226731         2022-06-01                        75   
3     7230744         2022-06-01                        91   
4     7289879         2022-06-01                        91   

     journey_start_town  journey_end_department journey_end_town  \
0                 Paris                      77         Jossigny   
1                 Paris                      77         Jossigny   
2                 Paris                      75            Paris   
3    Évry-Courcouronnes                      75            Paris   
4  Le Coudray-Montceaux                      75            Paris   

   passenger_seats  journey_distance  journey_duration  
0                1             25475                36  
1                1             25780                38  
2                1              3113           

### Saving to CSV

In [8]:
df_final.to_csv("registre-preuve-covoiturage_Data_Gouv.csv", sep=";", index=False)